In [1]:
# -*- coding: utf-8 -*-
# 공통 라이브러리 
import os
import time

# 토치 라이브러리 
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 그 외 필요한 머신러닝, 데이터 분석 라이브러리
import numpy as np
import pandas as pd
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from PIL import Image
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False
# 나눔고딕 폰트 적용 - 폰트 없을 시 https://dsstudy.tistory.com/15 참고하여 설치 , 윈도우 or 맥일 경우 운영체제에 맞게 폰트 설정
plt.rcParams["font.family"] = 'NanumGothic'




In [2]:
data_dir = "./data(oversampling)"

def create_datasets(batch_size):
    
    train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 좌우반전 
    transforms.RandomVerticalFlip(),  # 상하반전 
    transforms.Resize((512, 512)),  # 알맞게 변경하세요 
    transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize([0.41272497,0.38753614,0.37988776], [0.19911055,0.19538455, 0.18590365])  #  정규화(normalization)
])


    # choose the training and test datasets
    train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)



    # trainning set 중 validation 데이터로 사용할 비율
    valid_size = 0.3

    # validation으로 사용할 trainning indices를 얻는다.
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # trainning, validation batch를 얻기 위한 sampler정의
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=4)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=4)
    

    return train_data, train_loader, valid_loader


In [3]:
train_data, train_loader, valid_loader = create_datasets(batch_size=32)

In [4]:
class_names = train_data.classes

In [5]:
device ='cpu'

model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 88)


model = model.to(device)


In [6]:

batch_size = 32
model.load_state_dict(torch.load('vl0.09_7_3.pt', map_location='cpu'))

<All keys matched successfully>

In [8]:

data_dir = "./rawdata/"

import pandas as pd

df_submmit = pd.read_csv("sample_submission.csv", encoding="cp949")
df_test = pd.read_csv("test_df.csv", encoding="cp949")

transform = transforms.Compose([
    transforms.Resize((512, 512)),  # 알맞게 변경하세요 
    transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize([0.41272497,0.38753614,0.37988776], [0.19911055,0.19538455, 0.18590365])  #  정규화(normalization)
])

# image = Image.open('1차 용접부 데이터(추가데이터)_최종/test/1/7.jpg').convert('RGB')
test = os.listdir('./rawdata/test/')

for i in range(len(test)):
    
    image = Image.open(f'./rawdata/test/{test[i]}').convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        index = df_test[df_test["file_name"] ==test[i]]["index"].item()
        df_submmit.replace({"index":index},class_names[preds],inplace=True)

        

df_submmit.to_csv("./cnn_submission.csv")

In [9]:
# 이미지명을 라벨로 하는거
class MyDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform
        
    def get_class_label(self, image_name):
        # your method here
        y = ...
        return y
        
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        x = Image.open(image_path)
        y = self.get_class_label(image_path.split('/')[-1])
        if self.transform is not None:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return len(self.image_paths)

In [11]:
data_dir = "./rawdata"

def create_datasets(batch_size):
    
    train_transform = transforms.Compose([
    transforms.Resize((512, 512)),  # 알맞게 변경하세요 
    transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize([0.41272497,0.38753614,0.37988776], [0.19911055,0.19538455, 0.18590365])  #  정규화(normalization)
])

    # choose the training and test datasets
    test_data = MyDataset(os.path.join(data_dir, 'test'), train_transform)

    test_loader = torch.utils.data.DataLoader(test_data,                                    
                                               num_workers=4)

    return test_data, test_loader
    